In [658]:
import pandas as pd
import plotly.express as px
import numpy as np
import geopandas as gpd
import datetime

## configurations

In [659]:
pd.options.display.max_columns=100

## Loading Data

In [660]:
#building simulation
simulatedBldgsGdf = gpd.read_file('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/buildings_for_dashboard_4326.geojson')
simulatedBldgsGdf['start_date'] = pd.to_datetime(simulatedBldgsGdf['start_date'])
simulatedBldgsGdf['end_date'] = pd.to_datetime(simulatedBldgsGdf['end_date'])
#agents track
agents_track_status = pd.read_csv('https://raw.githubusercontent.com/Shai2u/demographic_estimation_dashboard_article/main/dashboard/data/agents_track_status.csv')
agents_track_status.drop(columns='Unnamed: 0',inplace=True)


## Prepearing date marks

In [661]:
year_makrs = [year for year in np.arange(2015,2031,0.5)]

In [662]:
years_with_q2_makrs = []
for year in year_makrs:
    d = pd.to_datetime(f'{int(year)}-07-01')
    if year % 1 == 0:
        d = pd.to_datetime(f'{int(year)}-01-01')
    years_with_q2_makrs.append(d)

In [663]:
d = np.random.randint(30)

In [664]:
random_date = years_with_q2_makrs[d]

In [665]:
print(random_date)

2020-07-01 00:00:00


In [666]:
bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< random_date ) & (simulatedBldgsGdf['end_date']> random_date )].copy().reset_index(drop=True)
bldgs1 = bldgs.copy()

In [667]:
bldgs1.head(2)

,fid,Area,floors,height,bld_num,bld_addres,hebrew_adr,project_nu,units,project_na,project_ty,status,East,North,East_wgs84,North_wgs8,active,start_date,end_date,GlobalID,deltaDays,Area_1,temp_heigh,randH,geometry
0,5.0,391.764026,3.0,13.0,212.0,208_12,"הלפר 12, בת ים",20181295,24.0,None,1.0,Building before,175911.943505,659490.282833,34.743557,32.027832,1.0,2000-01-01,2020-07-31,e7b90ee0-7820-4929-b1b4-2849aef8fdae,6848.0,546.669831,None,6,"POLYGON ((34.74367 32.02791, 34.74368 32.02775..."
1,13.0,346.811676,4.0,16.0,186.0,209_4,"ארלוזורוב 4, בת ים",20180845,20.0,None,1.0,Building before,175959.006193,659704.075843,34.744046,32.029762,1.0,2000-01-01,2020-10-18,de14ca13-2be5-47da-b7ae-9fe86d4eac63,6762.0,483.961366,None,11,"POLYGON ((34.74394 32.02972, 34.74399 32.02985..."


## Get income categories

In [668]:
incomeDict = {9000:'Low',19500:'Medium', 1000000:'High'}

In [669]:
incomeDict

{9000: 'Low', 19500: 'Medium', 1000000: 'High'}

In [671]:
incomeDict = {9000:'Low',19500:'Medium', 1000000:'High'}
agents_track_status['income_cat'] = pd.cut(agents_track_status['income'], [0,9000,19500,1000000],right=True, labels=['Low','Medium','High'],ordered=True)

agents_track_status.loc[agents_track_status['rent']==1,'rent_own']='Rent'
agents_track_status.loc[agents_track_status['rent']==0,'rent_own']='Own'

In [674]:
len(agents_track_status)

5748

## Get current year (testing)

In [675]:
agents_track_status_current = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']],left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])

In [676]:
agents_track_status_current['status_x'].unique()

array(['stay', 'Leave', 'New Comers'], dtype=object)

In [677]:
agents_track_status_current.drop_duplicates().reset_index(drop=True,inplace=True)

In [678]:
agents_track_status_current.tail()

,bldCode,doorIndex,bldCodeDoorIndex,ProjNumber,aprtmentSize,yearsInBldg,age,lowDiscount,highDiscount,noDiscount,income,rent,own,agentID,prjectType,tic,status_x,Floors,MainCost,cityTax,CostForStaying,ratioCostForStaying,reason_leave,mortgage,rentPrice,bld_status,income_cat,rent_own,project_nu,status_y,start_date
3980,201_75,25,201_75_25,502-0368860,87,39,23,0,0,1,11882.0,0,1,2a6da818-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0368860,Building before,2000-01-01
3981,201_75,26,201_75_26,502-0368860,87,35,63,0,0,1,12324.0,0,1,2a6da82c-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0368860,Building before,2000-01-01
3982,201_75,27,201_75_27,502-0368860,87,20,62,1,0,0,8312.0,0,1,2a6da84a-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own,502-0368860,Building before,2000-01-01
3983,201_75,28,201_75_28,502-0368860,87,1,47,0,0,1,15317.0,0,1,2a6da85e-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0368860,Building before,2000-01-01
3984,201_75,29,201_75_29,502-0368860,87,23,53,0,1,0,5862.0,0,1,2a6da87c-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own,502-0368860,Building before,2000-01-01


## Calibrate Age

In [679]:
age_grown1 = random_date.year-years_with_q2_makrs[0].year
agents_track_status_current.loc[agents_track_status_current['status_x']=='stay','age'] = agents_track_status_current.loc[agents_track_status_current['status_x']=='stay','age'] + age_grown1

In [680]:
agents_track_status_current.loc[agents_track_status_current['status_x']=='New Comers','age']

36      54
37      59
38      55
39      59
40      46
        ..
2851    35
2852    35
2853    53
2854    42
2855    36
Name: age, Length: 323, dtype: int64

In [681]:
agents_track_status_current.loc[agents_track_status_current['status_x']=='New Comers','age'] = agents_track_status_current.loc[agents_track_status_current['status_x']=='New Comers'].apply(lambda p: p['age'] + (random_date.year - p['start_date'].year),axis=1)

In [682]:
income_dummies = pd.get_dummies(agents_track_status_current['income_cat'],prefix='income')

In [683]:
income_dummies = pd.get_dummies(agents_track_status_current['income_cat'],prefix='income')
agents_track_status_current = pd.concat([agents_track_status_current,income_dummies],axis=1)

In [684]:
agents_track_status_current

,bldCode,doorIndex,bldCodeDoorIndex,ProjNumber,aprtmentSize,yearsInBldg,age,lowDiscount,highDiscount,noDiscount,income,rent,own,agentID,prjectType,tic,status_x,Floors,MainCost,cityTax,CostForStaying,ratioCostForStaying,reason_leave,mortgage,rentPrice,bld_status,income_cat,rent_own,project_nu,status_y,start_date,income_Low,income_Medium,income_High
0,210_19,1,210_19_1,20180988,70,32,24,0,1,0,5145.0,0,1,2a6d0426-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own,20180988,Building before,2000-01-01,1,0,0
1,210_19,2,210_19_2,20180988,70,30,72,0,0,1,13088.0,0,1,2a6d04ee-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,20180988,Building before,2000-01-01,0,1,0
2,210_19,3,210_19_3,20180988,70,16,43,0,0,1,15361.0,0,1,2a6d052a-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,20180988,Building before,2000-01-01,0,1,0
3,210_19,4,210_19_4,20180988,70,6,21,0,1,0,5428.0,0,1,2a6d0566-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own,20180988,Building before,2000-01-01,1,0,0
4,210_19,5,210_19_5,20180988,70,17,66,0,0,1,12282.0,1,0,2a6d058e-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Rent,20180988,Building before,2000-01-01,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3980,201_75,25,201_75_25,502-0368860,87,39,28,0,0,1,11882.0,0,1,2a6da818-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0368860,Building before,2000-01-01,0,1,0
3981,201_75,26,201_75_26,502-0368860,87,35,68,0,0,1,12324.0,0,1,2a6da82c-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0368860,Building before,2000-01-01,0,1,0
3982,201_75,27,201_75_27,502-0368860,87,20,67,1,0,0,8312.0,0,1,2a6da84a-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own,502-0368860,Building before,2000-01-01,1,0,0
3983,201_75,28,201_75_28,502-0368860,87,1,52,0,0,1,15317.0,0,1,2a6da85e-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0368860,Building before,2000-01-01,0,1,0


In [650]:
# Time Changing Stats
# mean Age q1 age q2 age a3 age
#beyond 65 below 65
# mean income q1 income q2 income q3 income
# income class and ownership
# income class configuration
# Year Dataset with:

#Number of owners who are high income, medium income, low income
#Number of renters who are high income, medium income, low income
# number o high income, medium income low income that are staying and new comers
#Average Age of people Staying q1 of people taying and q2, q3 of people styaing and leaving
# income of people staying and people leaving

In [651]:
yearly_stats = pd.DataFrame(columns=['year', 'New Comers_apartment_size_q1', 'stay_apartment_size_q1',
       'New Comers_apartment_size_q2', 'stay_apartment_size_q2',
       'New Comers_apartment_size_q3', 'stay_apartment_size_q3',
       'New Comers_age_q1', 'stay_age_q1', 'New Comers_age_q2', 'stay_age_q2',
       'New Comers_age_q3', 'stay_age_q3', 'New Comers_income_q1',
       'stay_income_q1', 'New Comers_income_q2', 'stay_income_q2',
       'New Comers_income_q3', 'stay_income_q3', 'New Comers_rent',
       'stay_rent', 'New Comers_own', 'stay_own', 'New Comers_income_low',
       'stay_income_low', 'New Comers_income_medium', 'stay_income_medium',
       'New Comers_income_high', 'stay_income_high'])

In [603]:
yearly_stats

,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high


In [604]:
step_1 = agents_track_status_current.groupby('status_x').agg({'aprtmentSize':[lambda p: np.quantile(p,0.25),lambda p: np.quantile(p,0.5),lambda p: np.quantile(p,0.75)],
                                                    'age':[lambda p: np.quantile(p,0.25),lambda p: np.quantile(p,0.5),lambda p: np.quantile(p,0.75)],
                                                    'income':[lambda p: np.quantile(p,0.25),lambda p: np.quantile(p,0.5),lambda p: np.quantile(p,0.75)],'rent':'count','own':'count','income_Low':'count','income_Medium':'count','income_High':'count'}).reset_index()
step_1 = step_1[step_1['status_x']!='Leave']
step_1.columns=['status','apartment_size_q1','apartment_size_q2','apartment_size_q3','age_q1','age_q2','age_q3','income_q1','income_q2','income_q3','rent','own','income_low','income_medium','income_high']
step_1['year'] = random_date.year
step_2 = step_1.pivot(columns='status',index='year')
step_2.columns = [f'{col[1]}_{col[0]}' for col in step_2.columns]
step_2.reset_index(inplace=True)

In [605]:
step_2

,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high
0,2025,93.0,93.0,98.0,98.0,108.0,98.0,33.0,39.0,46.0,51.0,62.0,57.0,22112.157895,7778.0,23844.144737,8954.0,25678.973684,12621.0,8660,584,8660,584,8660,584,8660,584,8660,584


In [606]:
year_date.month

7

In [652]:
for year_date in years_with_q2_makrs:

    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< year_date ) & (simulatedBldgsGdf['end_date']> year_date )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()
    #merge agents status and bldgs
    agents_track_status_current = pd.merge(agents_track_status,bldgs1[['project_nu','status','start_date']].drop_duplicates().reset_index(drop=True),left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])
    #calibrate age
    age_grown1 = year_date.year-years_with_q2_makrs[0].year
    agents_track_status_current.loc[agents_track_status_current['status_x']=='stay','age'] = agents_track_status_current.loc[agents_track_status_current['status_x']=='stay','age'] + age_grown1
    agents_track_status_current.loc[agents_track_status_current['status_x']=='New Comers','age'] = agents_track_status_current.loc[agents_track_status_current['status_x']=='New Comers'].apply(lambda p: p['age'] + (year_date.year - p['start_date'].year),axis=1)
    income_dummies = pd.get_dummies(agents_track_status_current['income_cat'],prefix='income')
    agents_track_status_current = pd.concat([agents_track_status_current,income_dummies],axis=1)
    step_1 = agents_track_status_current.groupby('status_x').agg({'aprtmentSize':[lambda p: np.quantile(p,0.25),lambda p: np.quantile(p,0.5),lambda p: np.quantile(p,0.75)],
                                                    'age':[lambda p: np.quantile(p,0.25),lambda p: np.quantile(p,0.5),lambda p: np.quantile(p,0.75)],
                                                    'income':[lambda p: np.quantile(p,0.25),lambda p: np.quantile(p,0.5),lambda p: np.quantile(p,0.75)],'rent':'sum','own':'sum','income_Low':'sum','income_Medium':'sum','income_High':'sum'}).reset_index()
    step_1 = step_1[step_1['status_x']!='Leave']
    step_1.columns=['status','apartment_size_q1','apartment_size_q2','apartment_size_q3','age_q1','age_q2','age_q3','income_q1','income_q2','income_q3','rent','own','income_low','income_medium','income_high']
    step_1['year'] = year_date.year
    if year_date.month==7:
        step_1['year'] = step_1['year'].astype(str) + ' Q3'
    else:
        step_1['year'] = step_1['year'].astype(str) + ' Q1'
        
    step_2 = step_1.pivot(columns='status',index='year')
    step_2.columns = [f'{col[1]}_{col[0]}' for col in step_2.columns]
    step_2.reset_index(inplace=True)
    yearly_stats = pd.concat([yearly_stats,step_2])

In [653]:
year_date = years_with_q2_makrs[0]

In [533]:
    bldgs  = simulatedBldgsGdf[(simulatedBldgsGdf['start_date']< year_date ) & (simulatedBldgsGdf['end_date']> year_date )].copy().reset_index(drop=True)
    bldgs1 = bldgs.copy()

In [654]:
agents_track_status.tail()

,bldCode,doorIndex,bldCodeDoorIndex,ProjNumber,aprtmentSize,yearsInBldg,age,lowDiscount,highDiscount,noDiscount,income,rent,own,agentID,prjectType,tic,status,Floors,MainCost,cityTax,CostForStaying,ratioCostForStaying,reason_leave,mortgage,rentPrice,bld_status,income_cat,rent_own
5743,201_75_rr,24,201_75_rr_24,502-0368860,112,0,50,0,0,1,25230.000000,0,1,3cb6769e-9b83-11eb-88c0-acde48001122,3,32,New Comers,32.0,450.0,608.16,9417.16,0.373252,NaN,8359,NaN,Building after,High,Own
5744,201_75_rr,25,201_75_rr_25,502-0368860,112,0,27,0,0,1,24886.684211,0,1,3cb67748-9b83-11eb-88c0-acde48001122,3,32,New Comers,32.0,450.0,608.16,9369.16,0.376473,NaN,8311,NaN,Building after,High,Own
5745,201_75_rr,26,201_75_rr_26,502-0368860,112,0,24,0,0,1,27497.210526,1,0,3cb6777a-9b83-11eb-88c0-acde48001122,3,32,New Comers,32.0,450.0,608.16,8898.16,0.323602,NaN,0,NaN,Building after,High,Rent
5746,201_75_rr,27,201_75_rr_27,502-0368860,112,0,39,0,0,1,27374.684211,0,1,3cb677ac-9b83-11eb-88c0-acde48001122,3,32,New Comers,32.0,450.0,608.16,9369.16,0.342256,NaN,8311,NaN,Building after,High,Own
5747,201_75_rr,29,201_75_rr_29,502-0368860,112,0,66,0,0,1,23963.210526,1,0,3cb677d4-9b83-11eb-88c0-acde48001122,3,32,New Comers,32.0,450.0,608.16,8898.16,0.371326,NaN,0,NaN,Building after,High,Rent


In [655]:
agents_track_status_current.tail()

,bldCode,doorIndex,bldCodeDoorIndex,ProjNumber,aprtmentSize,yearsInBldg,age,lowDiscount,highDiscount,noDiscount,income,rent,own,agentID,prjectType,tic,status_x,Floors,MainCost,cityTax,CostForStaying,ratioCostForStaying,reason_leave,mortgage,rentPrice,bld_status,income_cat,rent_own,project_nu,status_y,start_date,income_Low,income_Medium,income_High
3949,201_75_rr,24,201_75_rr_24,502-0368860,112,0,53,0,0,1,25230.000000,0,1,3cb6769e-9b83-11eb-88c0-acde48001122,3,32,New Comers,32.0,450.0,608.16,9417.16,0.373252,NaN,8359,NaN,Building after,High,Own,502-0368860,Building after,2027-05-07,0,0,1
3950,201_75_rr,25,201_75_rr_25,502-0368860,112,0,30,0,0,1,24886.684211,0,1,3cb67748-9b83-11eb-88c0-acde48001122,3,32,New Comers,32.0,450.0,608.16,9369.16,0.376473,NaN,8311,NaN,Building after,High,Own,502-0368860,Building after,2027-05-07,0,0,1
3951,201_75_rr,26,201_75_rr_26,502-0368860,112,0,27,0,0,1,27497.210526,1,0,3cb6777a-9b83-11eb-88c0-acde48001122,3,32,New Comers,32.0,450.0,608.16,8898.16,0.323602,NaN,0,NaN,Building after,High,Rent,502-0368860,Building after,2027-05-07,0,0,1
3952,201_75_rr,27,201_75_rr_27,502-0368860,112,0,42,0,0,1,27374.684211,0,1,3cb677ac-9b83-11eb-88c0-acde48001122,3,32,New Comers,32.0,450.0,608.16,9369.16,0.342256,NaN,8311,NaN,Building after,High,Own,502-0368860,Building after,2027-05-07,0,0,1
3953,201_75_rr,29,201_75_rr_29,502-0368860,112,0,69,0,0,1,23963.210526,1,0,3cb677d4-9b83-11eb-88c0-acde48001122,3,32,New Comers,32.0,450.0,608.16,8898.16,0.371326,NaN,0,NaN,Building after,High,Rent,502-0368860,Building after,2027-05-07,0,0,1


In [569]:
bldgs2 = bldgs1[['project_nu','status','start_date']].drop_duplicates().reset_index(drop=True)

In [611]:
bldgs2.tail()

,project_nu,status,start_date
32,502-0178285,Building before,2000-01-01
33,502-0317495,Building before,2000-01-01
34,502-0189720,Building before,2000-01-01
35,502-0316588,Building before,2000-01-01
36,502-0368860,Building before,2000-01-01


In [572]:
agents_track_status_current = pd.merge(agents_track_status,bldgs2,left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'],how='left')



In [580]:
agents_track_status[agents_track_status['bld_status']=='Building before']

,bldCode,doorIndex,bldCodeDoorIndex,ProjNumber,aprtmentSize,yearsInBldg,age,lowDiscount,highDiscount,noDiscount,income,rent,own,agentID,prjectType,tic,status,Floors,MainCost,cityTax,CostForStaying,ratioCostForStaying,reason_leave,mortgage,rentPrice,bld_status,income_cat,rent_own
0,210_19,1,210_19_1,20180988,70,32,19,0,1,0,5145.0,0,1,2a6d0426-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own
1,210_19,2,210_19_2,20180988,70,30,67,0,0,1,13088.0,0,1,2a6d04ee-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own
2,210_19,3,210_19_3,20180988,70,16,38,0,0,1,15361.0,0,1,2a6d052a-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own
3,210_19,4,210_19_4,20180988,70,6,16,0,1,0,5428.0,0,1,2a6d0566-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own
4,210_19,5,210_19_5,20180988,70,17,61,0,0,1,12282.0,1,0,2a6d058e-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Rent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5560,201_75,25,201_75_25,502-0368860,87,39,23,0,0,1,11882.0,0,1,2a6da818-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own
5561,201_75,26,201_75_26,502-0368860,87,35,63,0,0,1,12324.0,0,1,2a6da82c-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own
5562,201_75,27,201_75_27,502-0368860,87,20,62,1,0,0,8312.0,0,1,2a6da84a-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own
5563,201_75,28,201_75_28,502-0368860,87,1,47,0,0,1,15317.0,0,1,2a6da85e-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own


In [609]:
yearly_stats = yearly_stats.reset_index(drop=True)

In [656]:
yearly_stats

,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high
0,2015 Q1,NaN,66.0,NaN,73.0,NaN,83.0,NaN,30.0,NaN,42.0,NaN,65.0,NaN,6403.0,NaN,8771.0,NaN,11667.0,NaN,318,NaN,579,NaN,470.0,NaN,427.0,NaN,0
0,2015 Q3,NaN,68.0,NaN,73.0,NaN,83.0,NaN,30.0,NaN,42.0,NaN,65.0,NaN,6413.5,NaN,8804.0,NaN,11669.0,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0
0,2016 Q1,NaN,68.0,NaN,73.0,NaN,83.0,NaN,31.0,NaN,43.0,NaN,66.0,NaN,6413.5,NaN,8804.0,NaN,11669.0,NaN,310,NaN,565,NaN,454.0,NaN,421.0,NaN,0
0,2016 Q3,NaN,66.0,NaN,73.0,NaN,83.0,NaN,31.0,NaN,43.0,NaN,65.25,NaN,6427.0,NaN,8833.5,NaN,11729.75,NaN,294,NaN,542,NaN,432.0,NaN,404.0,NaN,0
0,2017 Q1,82.0,66.0,82.0,73.0,82.0,83.0,32.0,32.0,47.0,44.0,60.0,66.75,18838.105263,6496.0,21097.105263,8934.5,22033.947368,11806.25,11,282,14,520,0.0,405.0,11.0,397.0,14,0
0,2017 Q3,82.0,66.0,82.0,73.0,82.0,83.0,32.0,32.0,47.0,44.0,60.0,66.75,18838.105263,6496.0,21097.105263,8934.5,22033.947368,11806.25,11,282,14,520,0.0,405.0,11.0,397.0,14,0
0,2018 Q1,82.0,66.0,82.0,73.0,92.0,83.0,31.75,33.0,42.0,45.0,60.25,68.0,19619.105263,6497.0,21099.868421,8933.0,22017.447368,11867.5,31,278,53,517,0.0,402.0,20.0,393.0,64,0
0,2018 Q3,72.0,66.0,82.0,73.0,92.0,83.0,31.0,33.0,44.0,45.0,59.0,68.0,18584.605263,6497.0,20230.421053,8933.0,22263.842105,11867.5,69,278,134,517,0.0,402.0,75.0,393.0,128,0
0,2019 Q1,82.0,66.0,82.0,73.0,102.0,80.0,32.0,34.0,46.0,46.5,62.0,69.0,19415.105263,6503.75,21815.473684,8965.0,25173.131579,11886.75,90,271,195,493,0.0,384.0,75.0,380.0,210,0
0,2019 Q3,82.0,66.0,82.0,73.0,102.0,80.0,32.0,34.0,46.0,46.5,62.0,69.0,19415.105263,6503.75,21815.473684,8965.0,25173.131579,11886.75,90,271,195,493,0.0,384.0,75.0,380.0,210,0


In [657]:
yearly_stats.to_csv('yearly_stats_for_dashboard.csv')

In [497]:
step_2

,year,New Comers_apartment_size_q1,stay_apartment_size_q1,New Comers_apartment_size_q2,stay_apartment_size_q2,New Comers_apartment_size_q3,stay_apartment_size_q3,New Comers_age_q1,stay_age_q1,New Comers_age_q2,stay_age_q2,New Comers_age_q3,stay_age_q3,New Comers_income_q1,stay_income_q1,New Comers_income_q2,stay_income_q2,New Comers_income_q3,stay_income_q3,New Comers_rent,stay_rent,New Comers_own,stay_own,New Comers_income_low,stay_income_low,New Comers_income_medium,stay_income_medium,New Comers_income_high,stay_income_high
0,2030,93.0,93.0,98.0,98.0,108.0,98.0,38.0,44.0,51.0,56.0,67.0,62.0,22112.157895,7778.0,23844.144737,8954.0,25678.973684,12621.0,8660,584,8660,584,8660,584,8660,584,8660,584


In [493]:
year_date

Timestamp('2029-07-01 00:00:00')

In [491]:
years_with_q2_makrs

[Timestamp('2015-01-01 00:00:00'),
 Timestamp('2015-07-01 00:00:00'),
 Timestamp('2016-01-01 00:00:00'),
 Timestamp('2016-07-01 00:00:00'),
 Timestamp('2017-01-01 00:00:00'),
 Timestamp('2017-07-01 00:00:00'),
 Timestamp('2018-01-01 00:00:00'),
 Timestamp('2018-07-01 00:00:00'),
 Timestamp('2019-01-01 00:00:00'),
 Timestamp('2019-07-01 00:00:00'),
 Timestamp('2020-01-01 00:00:00'),
 Timestamp('2020-07-01 00:00:00'),
 Timestamp('2021-01-01 00:00:00'),
 Timestamp('2021-07-01 00:00:00'),
 Timestamp('2022-01-01 00:00:00'),
 Timestamp('2022-07-01 00:00:00'),
 Timestamp('2023-01-01 00:00:00'),
 Timestamp('2023-07-01 00:00:00'),
 Timestamp('2024-01-01 00:00:00'),
 Timestamp('2024-07-01 00:00:00'),
 Timestamp('2025-01-01 00:00:00'),
 Timestamp('2025-07-01 00:00:00'),
 Timestamp('2026-01-01 00:00:00'),
 Timestamp('2026-07-01 00:00:00'),
 Timestamp('2027-01-01 00:00:00'),
 Timestamp('2027-07-01 00:00:00'),
 Timestamp('2028-01-01 00:00:00'),
 Timestamp('2028-07-01 00:00:00'),
 Timestamp('2029-01-

In [547]:
bldgs1

,fid,Area,floors,height,bld_num,bld_addres,hebrew_adr,project_nu,units,project_na,project_ty,status,East,North,East_wgs84,North_wgs8,active,start_date,end_date,GlobalID,deltaDays,Area_1,temp_heigh,randH,geometry
0,1.0,456.624954,3.0,13.0,94.0,215_22,"סוקולוב 22, בת ים",20140882,18.0,None,1.0,Building before,175757.206152,658982.882610,34.741942,32.023251,1.0,2000-01-01,2016-06-02,15067677-2690-4821-9de9-16bde042f745,5322.0,637.113372,None,5,"POLYGON ((34.74172 32.02321, 34.74172 32.02327..."
1,2.0,265.821715,3.0,13.0,17.0,216_4,"מסריק 4, בת ים",20141086,12.0,None,1.0,Building before,175680.060156,659070.929640,34.741121,32.024042,1.0,2000-01-01,2015-06-17,374f9596-2397-433c-96d6-82ec7f214154,5394.0,370.895857,None,4,"POLYGON ((34.74107 32.02401, 34.74107 32.02401..."
2,3.0,158.986180,3.0,13.0,214.0,209_6,"ארלוזורוב 6, בת ים",20140231,6.0,None,2.0,Building before,175950.936300,659680.635944,34.743961,32.029550,1.0,2000-01-01,2016-07-28,b662cfa9-66ce-4e9c-ba55-4474171e5ff6,5130.0,221.860645,None,5,"POLYGON ((34.74388 32.02952, 34.74391 32.02961..."
3,4.0,743.350688,4.0,16.0,213.0,209_8_209_10_209_12,"ארלוזורוב 8-12, בת ים",20120894,48.0,None,1.0,Building before,175941.019136,659640.557221,34.743858,32.029188,1.0,2000-01-01,2015-06-02,12a97223-90b2-47db-8dc4-a179b2a5ed33,4768.0,1037.304708,None,4,"POLYGON ((34.74373 32.02907, 34.74373 32.02907..."
4,5.0,391.764026,3.0,13.0,212.0,208_12,"הלפר 12, בת ים",20181295,24.0,None,1.0,Building before,175911.943505,659490.282833,34.743557,32.027832,1.0,2000-01-01,2020-07-31,e7b90ee0-7820-4929-b1b4-2849aef8fdae,6848.0,546.669831,None,6,"POLYGON ((34.74367 32.02791, 34.74368 32.02775..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,67.0,915.761705,3.0,13.0,323.0,221_4_221_2,"בלפור 2-4, בת ים",502-0316588,40.0,NULLבי/ 514 פינוי בינוי 'שער העיר הצפוני',3.0,Building before,175921.475026,659972.213328,34.743636,32.032178,1.0,2000-01-01,2021-02-17,77175586-b391-4c0f-bd8e-5e75e88a6951,5467.0,1277.962939,None,9,"POLYGON ((34.74340 32.03221, 34.74360 32.03234..."
67,68.0,544.182041,3.0,13.0,324.0,221_6,"בלפור 6, בת ים",502-0316588,20.0,NULLבי/ 514 פינוי בינוי 'שער העיר הצפוני',3.0,Building before,175921.992539,659924.656454,34.743644,32.031750,1.0,2000-01-01,2021-02-17,193e50f8-72ea-4467-8125-1462c315e6cf,5467.0,759.416518,None,8,"POLYGON ((34.74352 32.03175, 34.74352 32.03175..."
68,69.0,305.907044,2.0,10.0,380.0,201_75,"שדרות העצמאות 75, בת ים",502-0368860,1.0,בי/1/466/מק,3.0,Building before,176428.017768,659582.676773,34.749016,32.028685,1.0,2000-01-01,2024-04-22,75bf4cc3-978f-4519-9ab1-b294acb2889c,6507.0,426.865804,None,6,"POLYGON ((34.74901 32.02857, 34.74899 32.02858..."
69,70.0,0.000000,3.0,13.0,3.0,216_8,"מסריק 10, בת ים",502-0654715,18.0,בי/565 - חוף הצוק,3.0,Building before,175667.533496,659031.128966,34.740991,32.023682,1.0,2000-01-01,2024-05-17,f464c1de-f810-4e18-bacb-8318619180f6,6592.0,487.859725,None,6,"POLYGON ((34.74085 32.02367, 34.74085 32.02367..."


In [ ]:
agents_track_status[agents_track_status['bld_status']=='Building before']

In [ ]:
agents_track_status[agents_track_status['bld_status']=='Building before']

In [560]:
agents_track_status['ProjNumber'].value_counts()

502-0196659    1110
502-0178285     470
502-0654715     470
502-0317495     413
502-0316588     274
502-0201483     267
502-0189720     262
502-0147678     254
502-0368860     212
20180345        156
502-0424911     152
20160319        130
20170076        119
20181295        119
20120894        109
20180845         95
502-0583948      95
20180988         93
20110270         86
20180812         83
20140556         78
20171974         78
502-0316752      76
20150274         74
20181092         74
20100133         73
20150857         73
20140060         72
20140467         71
20141182         64
20140231         46
Name: ProjNumber, dtype: int64

In [548]:
agents_track_status['bld_status'].value_counts()

Building after     3954
Building before     897
Construction        897
Name: bld_status, dtype: int64

In [562]:
projNumber='502-0196659'
at_pn = agents_track_status[agents_track_status['ProjNumber']==projNumber].reset_index()

In [563]:
at_pn.tail()

,index,bldCode,doorIndex,bldCodeDoorIndex,ProjNumber,aprtmentSize,yearsInBldg,age,lowDiscount,highDiscount,noDiscount,income,rent,own,agentID,prjectType,tic,status,Floors,MainCost,cityTax,CostForStaying,ratioCostForStaying,reason_leave,mortgage,rentPrice,bld_status,income_cat,rent_own
1105,1859,210_30_210_32_rr,56,210_30_210_32_rr_56,502-0196659,98,0,33,0,0,1,23876.368421,0,1,3cb44482-9b83-11eb-88c0-acde48001122,3,9,New Comers,30.0,450.0,532.14,8284.14,0.346960,NaN,7302,NaN,Building after,High,Own
1106,1860,210_30_210_32_rr,57,210_30_210_32_rr_57,502-0196659,98,0,65,0,0,1,26576.315789,0,1,3cb44586-9b83-11eb-88c0-acde48001122,3,9,New Comers,30.0,450.0,532.14,8238.14,0.309981,NaN,7256,NaN,Building after,High,Own
1107,1861,210_30_210_32_rr,58,210_30_210_32_rr_58,502-0196659,98,0,21,0,0,1,22912.315789,0,1,3cb445c2-9b83-11eb-88c0-acde48001122,3,9,New Comers,30.0,450.0,532.14,8257.14,0.360380,NaN,7275,NaN,Building after,High,Own
1108,1862,210_30_210_32_rr,59,210_30_210_32_rr_59,502-0196659,98,0,55,0,0,1,21552.210526,1,0,3cb445f4-9b83-11eb-88c0-acde48001122,3,9,New Comers,30.0,450.0,532.14,7842.14,0.363867,NaN,0,NaN,Building after,High,Rent
1109,1863,210_30_210_32_rr,60,210_30_210_32_rr_60,502-0196659,98,0,73,0,0,1,24704.315789,0,1,3cb4461c-9b83-11eb-88c0-acde48001122,3,9,New Comers,30.0,450.0,532.14,8238.14,0.333470,NaN,7256,NaN,Building after,High,Own


In [626]:
bldgs1[['project_nu','status','start_date']].value_counts()

project_nu   status           start_date
502-0196659  Building before  2000-01-01    11
502-0317495  Building before  2000-01-01     5
502-0178285  Building before  2000-01-01     4
502-0201483  Building before  2000-01-01     4
502-0316752  Building before  2000-01-01     4
502-0654715  Building before  2000-01-01     4
502-0424911  Building before  2000-01-01     4
502-0189720  Building before  2000-01-01     3
20170076     Building before  2000-01-01     3
502-0316588  Building before  2000-01-01     2
502-0368860  Building before  2000-01-01     1
20180812     Building before  2000-01-01     1
502-0583948  Building before  2000-01-01     1
502-0147678  Building before  2000-01-01     1
20181295     Building before  2000-01-01     1
20181114     Building before  2000-01-01     1
20181092     Building before  2000-01-01     1
20180988     Building before  2000-01-01     1
20180845     Building before  2000-01-01     1
20100133     Construction     2015-06-08     1
20110270     Buildi

Building before    68
Construction        3
Name: status, dtype: int64

In [565]:
pd.merge(at_pn,bldgs1[['project_nu','status','start_date']],how='inner',left_on=['ProjNumber','bld_status'], right_on=['project_nu','status'])

,index,bldCode,doorIndex,bldCodeDoorIndex,ProjNumber,aprtmentSize,yearsInBldg,age,lowDiscount,highDiscount,noDiscount,income,rent,own,agentID,prjectType,tic,status_x,Floors,MainCost,cityTax,CostForStaying,ratioCostForStaying,reason_leave,mortgage,rentPrice,bld_status,income_cat,rent_own,project_nu,status_y,start_date
0,754,224_3,1,224_3_1,502-0196659,73,34,68,0,1,1,11389.0,0,1,2a6d1e02-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0196659,Building before,2000-01-01
1,754,224_3,1,224_3_1,502-0196659,73,34,68,0,1,1,11389.0,0,1,2a6d1e02-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0196659,Building before,2000-01-01
2,754,224_3,1,224_3_1,502-0196659,73,34,68,0,1,1,11389.0,0,1,2a6d1e02-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0196659,Building before,2000-01-01
3,754,224_3,1,224_3_1,502-0196659,73,34,68,0,1,1,11389.0,0,1,2a6d1e02-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0196659,Building before,2000-01-01
4,754,224_3,1,224_3_1,502-0196659,73,34,68,0,1,1,11389.0,0,1,2a6d1e02-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Medium,Own,502-0196659,Building before,2000-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,953,224_7,20,224_7_20,502-0196659,73,32,20,1,0,0,8385.0,0,1,2a6d4b66-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own,502-0196659,Building before,2000-01-01
2196,953,224_7,20,224_7_20,502-0196659,73,32,20,1,0,0,8385.0,0,1,2a6d4b66-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own,502-0196659,Building before,2000-01-01
2197,953,224_7,20,224_7_20,502-0196659,73,32,20,1,0,0,8385.0,0,1,2a6d4b66-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own,502-0196659,Building before,2000-01-01
2198,953,224_7,20,224_7_20,502-0196659,73,32,20,1,0,0,8385.0,0,1,2a6d4b66-9b6a-11eb-ab03-acde48001122,0,0,stay,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,Building before,Low,Own,502-0196659,Building before,2000-01-01


,fid,Area,floors,height,bld_num,bld_addres,hebrew_adr,project_nu,units,project_na,project_ty,status,East,North,East_wgs84,North_wgs8,active,start_date,end_date,GlobalID,deltaDays,Area_1,temp_heigh,randH,geometry
0,1.0,456.624954,3.0,13.0,94.0,215_22,"סוקולוב 22, בת ים",20140882,18.0,None,1.0,Building before,175757.206152,658982.882610,34.741942,32.023251,1.0,2000-01-01,2016-06-02,15067677-2690-4821-9de9-16bde042f745,5322.0,637.113372,None,5,"POLYGON ((34.74172 32.02321, 34.74172 32.02327..."
1,2.0,265.821715,3.0,13.0,17.0,216_4,"מסריק 4, בת ים",20141086,12.0,None,1.0,Building before,175680.060156,659070.929640,34.741121,32.024042,1.0,2000-01-01,2015-06-17,374f9596-2397-433c-96d6-82ec7f214154,5394.0,370.895857,None,4,"POLYGON ((34.74107 32.02401, 34.74107 32.02401..."
2,3.0,158.986180,3.0,13.0,214.0,209_6,"ארלוזורוב 6, בת ים",20140231,6.0,None,2.0,Building before,175950.936300,659680.635944,34.743961,32.029550,1.0,2000-01-01,2016-07-28,b662cfa9-66ce-4e9c-ba55-4474171e5ff6,5130.0,221.860645,None,5,"POLYGON ((34.74388 32.02952, 34.74391 32.02961..."
3,4.0,743.350688,4.0,16.0,213.0,209_8_209_10_209_12,"ארלוזורוב 8-12, בת ים",20120894,48.0,None,1.0,Building before,175941.019136,659640.557221,34.743858,32.029188,1.0,2000-01-01,2015-06-02,12a97223-90b2-47db-8dc4-a179b2a5ed33,4768.0,1037.304708,None,4,"POLYGON ((34.74373 32.02907, 34.74373 32.02907..."
4,5.0,391.764026,3.0,13.0,212.0,208_12,"הלפר 12, בת ים",20181295,24.0,None,1.0,Building before,175911.943505,659490.282833,34.743557,32.027832,1.0,2000-01-01,2020-07-31,e7b90ee0-7820-4929-b1b4-2849aef8fdae,6848.0,546.669831,None,6,"POLYGON ((34.74367 32.02791, 34.74368 32.02775..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,67.0,915.761705,3.0,13.0,323.0,221_4_221_2,"בלפור 2-4, בת ים",502-0316588,40.0,NULLבי/ 514 פינוי בינוי 'שער העיר הצפוני',3.0,Building before,175921.475026,659972.213328,34.743636,32.032178,1.0,2000-01-01,2021-02-17,77175586-b391-4c0f-bd8e-5e75e88a6951,5467.0,1277.962939,None,9,"POLYGON ((34.74340 32.03221, 34.74360 32.03234..."
67,68.0,544.182041,3.0,13.0,324.0,221_6,"בלפור 6, בת ים",502-0316588,20.0,NULLבי/ 514 פינוי בינוי 'שער העיר הצפוני',3.0,Building before,175921.992539,659924.656454,34.743644,32.031750,1.0,2000-01-01,2021-02-17,193e50f8-72ea-4467-8125-1462c315e6cf,5467.0,759.416518,None,8,"POLYGON ((34.74352 32.03175, 34.74352 32.03175..."
68,69.0,305.907044,2.0,10.0,380.0,201_75,"שדרות העצמאות 75, בת ים",502-0368860,1.0,בי/1/466/מק,3.0,Building before,176428.017768,659582.676773,34.749016,32.028685,1.0,2000-01-01,2024-04-22,75bf4cc3-978f-4519-9ab1-b294acb2889c,6507.0,426.865804,None,6,"POLYGON ((34.74901 32.02857, 34.74899 32.02858..."
69,70.0,0.000000,3.0,13.0,3.0,216_8,"מסריק 10, בת ים",502-0654715,18.0,בי/565 - חוף הצוק,3.0,Building before,175667.533496,659031.128966,34.740991,32.023682,1.0,2000-01-01,2024-05-17,f464c1de-f810-4e18-bacb-8318619180f6,6592.0,487.859725,None,6,"POLYGON ((34.74085 32.02367, 34.74085 32.02367..."


In [617]:
bldgs1[['project_nu','status','start_date']].apply(lambda p: f"{p['project_nu']}_{p['status']}_{['start_date']}",axis=1)

0        20110270_Building after_['start_date']
1        20120894_Building after_['start_date']
2        20131419_Building after_['start_date']
3        20140060_Building after_['start_date']
4        20140231_Building after_['start_date']
5        20140467_Building after_['start_date']
6        20140556_Building after_['start_date']
7        20140882_Building after_['start_date']
8        20160319_Building after_['start_date']
9        20141190_Building after_['start_date']
10       20100133_Building after_['start_date']
11       20141086_Building after_['start_date']
12       20141182_Building after_['start_date']
13       20150274_Building after_['start_date']
14       20150857_Building after_['start_date']
15       20170076_Building after_['start_date']
16       20170076_Building after_['start_date']
17       20170076_Building after_['start_date']
18       20180845_Building after_['start_date']
19       20181295_Building after_['start_date']
20       20181092_Building after_['start